In [ ]:
#this code can be moved into a python script in the future, for now testing in a notebook

In [1]:
#imports
import pandas as pd
from nltk.corpus import stopwords

In [2]:
#get data - store in a zip file and open here, downloaded from the CFPB database. 
#saved this file locally as it is too large for Github, can do the same on your machines if you want to run
df = pd.read_csv("complaints.csv.zip")

FileNotFoundError: [Errno 2] No such file or directory: 'complaints.csv.zip'

In [ ]:
df.head()

In [3]:
#drop rows w/o a consumer complaint narrative (consistent with EDA)
df.dropna(subset = ['Consumer complaint narrative'], inplace=True)

NameError: name 'df' is not defined

In [23]:
#consistent w/ EDA analysis
new_names = [n.lower().replace(" ", "_").replace("?", "") for n in df.columns]
df.columns = new_names

In [24]:
df.head()

,date_received,product,sub-product,issue,sub-issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
11,2019-09-08,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Fraud or scam,NaN,"I was sold access to an event digitally, of wh...",NaN,"Paypal Holdings, Inc",RI,029XX,NaN,Consent provided,Web,2019-09-08,Closed with explanation,Yes,NaN,3366475
12,2019-09-24,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,While checking my credit report I noticed thre...,Company has responded to the consumer and the ...,"HCFS Health Care Financial Services, Inc.",FL,333XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3385399


In [25]:
df.dtypes

date_received                   object
product                         object
sub-product                     object
issue                           object
sub-issue                       object
consumer_complaint_narrative    object
company_public_response         object
company                         object
state                           object
zip_code                        object
tags                            object
consumer_consent_provided       object
submitted_via                   object
date_sent_to_company            object
company_response_to_consumer    object
timely_response                 object
consumer_disputed               object
complaint_id                     int64
dtype: object

In [26]:
df.describe(include='all')

,date_received,product,sub-product,issue,sub-issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
count,525657,525657,473485,525657,393192,525657,257386,525657,523645,407773,90154,525657,525657,525657,525656,525657,164066,5.256570e+05
unique,1871,18,75,161,216,493044,10,4519,63,11174,3,1,1,1872,6,2,2,NaN
top,2017-09-08,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,There are many mistakes appear in my report wi...,Company has responded to the consumer and the ...,"EQUIFAX, INC.",CA,300XX,Servicemember,Consent provided,Web,2017-09-08,Closed with explanation,Yes,No,NaN
freq,1890,161981,158449,90671,39972,1035,187629,57147,70210,9290,54595,525657,525657,1783,425542,511796,128259,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.693161e+06
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.190548e+05
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.290155e+06
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.263759e+06
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.810377e+06
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.195571e+06


Steps:
1) convert to lowercase
2) remove special characters and tokenize them into terms
3) remove stop words
4) stemming 
5) construct term-document matrix

Steps based on file:///Users/norahajjar/Downloads/Latent_Dirichlet_Allocation_LDA_for_Topic_Modeling.pdf
Cleaning code based on: https://ashwin-ks.github.io/2018-08-15-NLP-Consumer-Complaints-Classification-ML-DL/


In [27]:
# following text cleaning based on tutorial: https://ashwin-ks.github.io/2018-08-15-NLP-Consumer-Complaints-Classification-ML-DL/
#1) convert to lowercase
df['consumer_complaint_narrative'] =df['consumer_complaint_narrative'].apply(lambda x: ' '.join([i.lower() for i in x.split()]))



In [28]:
# remove punctuation
df['consumer_complaint_narrative'] =df['consumer_complaint_narrative'].str.replace(r'[^\w\s]',"")


In [46]:
#standardize text
#use external normalization dictionaries
#emnlp_dict.txt from http://people.eng.unimelb.edu.au/tbaldwin/etc/emnlp2012-lexnorm.tgz
#http://luululu.com/tweet/typo-corpus-r1.txt
dico = {}
dico1 = open('norm_dicts/emnlp_dict.txt', 'rb')
for word in dico1:
    word = word.decode('utf8')
    word = word.split()
    dico[word[0]] = word[1]
dico1.close()
dico2 = open('norm_dicts/typo-corpus-r1.txt', 'rb')
for word in dico2:
    word = word.decode('utf8')
    word = word.split()
    dico[word[0]] = word[1]
dico2.close()

In [48]:
def txt_std(words):
    list_words = words.split()
    for i in range(len(list_words)):
        if list_words[i] in dico.keys():
            list_words[i] = dico[list_words[i]]
    return ' '.join(list_words)

In [ ]:
#apply text standardization to text
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].apply(txt_std)


In [50]:
#remove redacted/x'ed out terms
df['consumer_complaint_narrative'] = df['consumer_complaint_narrative'].str.replace(r"xx+\s","")


In [51]:
df.consumer_complaint_narrative.head(1)

0    transworld systems inc is trying to collect a ...
Name: consumer_complaint_narrative, dtype: object

In [47]:
# 2) remove special characters and tokenize them into terms

In [ ]:
# 3) remove stop words
stop = stopwords.words('english')
df['consumer_complaint_narrative'] =df['consumer_complaint_narrative'].apply(lambda x: ' '.join([i for i in x.split() if i not in stop]))


In [18]:
df.head()

,date_received,product,sub-product,issue,sub-issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworldsystemsincistryingtocollectadebtthat...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,iwouldliketorequestthesuppressionofthefollowin...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,overthepast2weeksihavebeenreceivingexcessiveam...,NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
11,2019-09-08,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Fraud or scam,NaN,iwassoldaccesstoaneventdigitallyofwhichihaveal...,NaN,"Paypal Holdings, Inc",RI,029XX,NaN,Consent provided,Web,2019-09-08,Closed with explanation,Yes,NaN,3366475
12,2019-09-24,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,whilecheckingmycreditreportinoticedthreecollec...,Company has responded to the consumer and the ...,"HCFS Health Care Financial Services, Inc.",FL,333XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3385399


In [16]:
df.head()

,date_received,product,sub-product,issue,sub-issue,consumer_complaint_narrative,company_public_response,company,state,zip_code,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed,complaint_id
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc is trying to collect a ...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
2,2019-10-25,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,i would like to request the suppression of the...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,937XX,NaN,Consent provided,Web,2019-10-25,Closed with explanation,Yes,NaN,3417821
3,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,over the past 2 weeks i have been receiving ex...,NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
11,2019-09-08,"Money transfer, virtual currency, or money ser...",Domestic (US) money transfer,Fraud or scam,NaN,i was sold access to an event digitally of whi...,NaN,"Paypal Holdings, Inc",RI,029XX,NaN,Consent provided,Web,2019-09-08,Closed with explanation,Yes,NaN,3366475
12,2019-09-24,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,while checking my credit report i noticed thre...,Company has responded to the consumer and the ...,"HCFS Health Care Financial Services, Inc.",FL,333XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3385399
